# nbgrader workflow

Let op: voor een correcte werking moet deze workflow in een eigen map uitgevoerd worden. Dit notebook moet in deze map staan en Jupyter moet vanuit deze map opgestart te worden. Dit is een beperking van nbgrader.

In [1]:
import os
import re
import shutil
from glob import glob
import pandas as pd

from IPython.display import display, Markdown, Javascript

display(Javascript("if(!(window.location.pathname.indexOf('notebooks') == 1 && window.location.pathname.split('/').length == 3)) { alert('De workflow-notebook staat niet in de root-map van Jupyter!'); }"))

<IPython.core.display.Javascript object>

## 0. Configuratie

In [2]:
course_id = "DataScience"  # Name of the course
assignment_id = "Deeltentamen1"   # Name of the exam

## 1. nbgrader configureren
code-stub overschrijven etc.

In [3]:
config = '''
c = get_config()

c.Exchange.course_id = "%s"
c.ClearSolutions.code_stub = {'python': '# JOUW CODE HIER', 'markdown': '....'}
''' % course_id

with open('nbgrader_config.py', 'w') as f:
    f.write(config)

## 2. Assignment-map aanmaken

* You create a `source` folder with a folder with the name of the assignment.
* A template exam is copied to that folder.

In [18]:
!nbgrader db assignment add {assignment_id}
os.makedirs('source/%s' % assignment_id, exist_ok=True)


nbgrader_template='https://raw.githubusercontent.com/UvA-FNWI/notebook-exams/master/Using_notebooks_for_exams/Using_nbgrader/nbgrader-template.ipynb'
import urllib.request
urllib.request.urlretrieve(nbgrader_template, 'source/%s/%s.ipynb' % (assignment_id, assignment_id))

!ls -l source/{assignment_id}

display(Markdown('---\n## 3. Assignment maken\nHet notebook-template staat in **source/%s/%s.ipynb**  \n<a class="btn btn-primary" style="margin-top: 10px; text-decoration: none;" href="/notebooks/source/%s/%s.ipynb" target="_blank">Open notebook</a>' % (assignment_id, assignment_id, assignment_id, assignment_id)))

[DbAssignmentAddApp | INFO] Creating/updating assignment with ID 'Deeltentamen1': {'duedate': None}
total 24
-rw-r--r--@ 1 Jesse  staff  8526 Apr 12 14:40 Deeltentamen1.ipynb


---
## 3. Assignment maken
Het notebook-template staat in **source/Deeltentamen1/Deeltentamen1.ipynb**  
<a class="btn btn-primary" style="margin-top: 10px; text-decoration: none;" href="/notebooks/source/Deeltentamen1/Deeltentamen1.ipynb" target="_blank">Open notebook</a>

nbgrader documentatie: http://nbgrader.readthedocs.io/en/stable/user_guide/creating_and_grading_assignments.html

## 4. Maak een studentversie

In [8]:
shutil.rmtree('release', ignore_errors=True)

!nbgrader update ./source/

!nbgrader assign {assignment_id} --create

student_nb = glob('release/%s/*.ipynb' % assignment_id)[0]
student_nb_filename = os.path.basename(student_nb)
student_nb_name = os.path.splitext(student_nb_filename)[0]

display(Markdown('---\nDe studentversie bevindt zich in **%s**  \n<a class="btn btn-primary" style="margin-top: 10px; text-decoration: none;" href="/notebooks/%s" target="_blank">Open notebook</a>' % (student_nb, student_nb)))

[UpdateApp | INFO] Updating metadata for notebook: ./source/Deeltentamen1/Deeltentamen1.ipynb
[AssignApp | INFO] Updating/creating assignment 'Deeltentamen1': {}
[AssignApp | INFO] Converting notebook /Users/Jesse/Dropbox/Notebook-exams/nbgrader-template/source/./Deeltentamen1/Deeltentamen1.ipynb
[AssignApp | INFO] Writing 7688 bytes to /Users/Jesse/Dropbox/Notebook-exams/nbgrader-template/release/./Deeltentamen1/Deeltentamen1.ipynb
[AssignApp | INFO] Setting destination file permissions to 644


---
De studentversie bevindt zich in **release/Deeltentamen1/Deeltentamen1.ipynb**  
<a class="btn btn-primary" style="margin-top: 10px; text-decoration: none;" href="/notebooks/release/Deeltentamen1/Deeltentamen1.ipynb" target="_blank">Open notebook</a>

## 5. Verzamel inzendingen

Oude versie, voor als de nbgrader zip collect niet werkt:

In [30]:
'''
submissions_zip = 'submissions.zip'

shutil.rmtree('original-submissions', ignore_errors=True)
!unzip -j {submissions_zip} -d "original-submissions"

nbs_copied = 0

for f in glob('original-submissions/*.ipynb'):
    nb = os.path.basename(f)
    
    ids = re.findall(r'(\d{7,8})\.ipynb', nb)
    if len(ids):
        student_id = ids[0]
        
        os.makedirs('submitted/%s/%s' % (student_id, assignment_id), exist_ok=True)
        shutil.copyfile(f, 'submitted/%s/%s/%s' % (student_id, assignment_id, student_nb_filename))
        
        nbs_copied += 1

display(Markdown('---\n%d notebook(s) gevonden en gekopiëerd  \n<a class="btn btn-primary" style="margin-top: 10px; text-decoration: none;" href="/tree/submitted" target="_blank">Open inzendingen-map</a>' % nbs_copied))
''';

In [ ]:
def clean_up_submissionszip(submission_zip=submissions_zip, 
                            assignment_id=assignment_id,
                            all_but_filename_regex= r'.* \d{7,8}_\d+_',
                            folder='effe'):
    '''This function cleans submissions.zip
 It removes files without a valid student id
 It renames all files to the original filename (assignment_id.ipynb)
    '''
    all_but_filename_regex= r'.* \d{7,8}_\d+_'


    !unzip -q $submissions_zip -d $folder
    !rm $submissions_zip
    for filename in os.listdir(folder):
        if re.search(all_but_filename_regex,filename):
            prefix= re.findall(all_but_filename_regex,filename)[0]
            os.rename(os.path.join(folder,filename), os.path.join(folder,prefix+assignment_id+'.ipynb') )
        else:
            os.remove(os.path.join(folder,filename))

    %cd $folder
    !zip -q ../$submissions_zip *
    %cd ..
    !rm -r $folder
    print('Cleaned %s by removing filenames without valid student id, and renaming all files to %s'%(submissions_zip,assignment_id))
    return True

In [31]:
# naam van de zip file  met de submissies (uit bijvoorbeeld testvision of Blackboard/canvas)
# Let op, de zip file mag geen folders bevatten
submissions_zip = 'submissions.zip'

# clean up submissions zip file
clean_up_submissionszip()


# maak de benodigde folders
makedirs('downloaded/%s/archive', exist_ok=True)
copyfile(submissions_zip, 'downloaded/%s/archive/' % assignment_id)

# Voorbeeld filenaam uit testvision: 
#  001_73352_Witting, B, 11003952_182493_DS_Deeltentamen1_2018.ipynb
# This regex yields:  ('11003952', 'DS_Deeltentamen1_2018.ipynb')

### Let op: het deel dat matched met de <file_id> groep moet gelijk zijn aan {assignment_id}.ipynb
# Regex voor TestVision
regex = ".*\s(?P<student_id>\d{7,8}).*?(?P<file_id>[^\d_]\w+\.ipynb)"

#regex = r'(?P<file_id>[^\d_]\w+)_(?P<student_id>\d{7,8})\.ipynb'

!nbgrader zip_collect {assignment_id} --force --FileNameCollectorPlugin.named_regexp="$regex"

display(Markdown('<a class="btn btn-primary" style="margin-top: 10px; text-decoration: none;" href="/tree/submitted" target="_blank">Open inzendingen-map</a>'))

[ZipCollectApp | INFO] Using file extractor: ExtractorPlugin
[ZipCollectApp | INFO] Using file collector: FileNameCollectorPlugin
[ZipCollectApp | WARNING] Clearing existing files in /Users/Jesse/Dropbox/Notebook-exams/nbgrader-template/downloaded/Deeltentamen1/extracted
[ZipCollectApp | INFO] Extracting from: /Users/Jesse/Dropbox/Notebook-exams/nbgrader-template/downloaded/Deeltentamen1/archive/submissions.zip
[ZipCollectApp | INFO]   Extracting to: /Users/Jesse/Dropbox/Notebook-exams/nbgrader-template/downloaded/Deeltentamen1/extracted/submissions
[ZipCollectApp | INFO] Start collecting files...
[ZipCollectApp | INFO] Parsing file: /Users/Jesse/Dropbox/Notebook-exams/nbgrader-template/downloaded/Deeltentamen1/extracted/submissions/__MACOSX/._submissions
[ZipCollectApp | WARNING] Skipped submission with no match information provided: /Users/Jesse/Dropbox/Notebook-exams/nbgrader-template/downloaded/Deeltentamen1/extracted/submissions/__MACOSX/._submissions
[ZipCollectApp | INFO] Parsin

<a class="btn btn-primary" style="margin-top: 10px; text-decoration: none;" href="/tree/submitted" target="_blank">Open inzendingen-map</a>

## 6. Autograden

In [35]:
!nbgrader autograde {assignment_id} --create

display(Markdown('---\n<a class="btn btn-primary" style="margin-top: 10px; text-decoration: none;" href="/formgrader/gradebook/%s/%s" target="_blank">Open manual grading</a>  \n\n_Als het klikken op een inzending een 404-error geeft kan het helpen om heel Jupyter Notebook opnieuw op te starten._' % (assignment_id, student_nb_name)))

[AutogradeApp | ERROR] No notebooks were matched by '/Users/Jesse/Dropbox/Notebook-exams/nbgrader-template/submitted/*/Deeltentamen1'


---
<a class="btn btn-primary" style="margin-top: 10px; text-decoration: none;" href="/formgrader/gradebook/Deeltentamen1/Deeltentamen1" target="_blank">Open manual grading</a>  

_Als het klikken op een inzending een 404-error geeft kan het helpen om heel Jupyter Notebook opnieuw op te starten._

## 7. students-answers.csv genereren

In [21]:
import numpy as np

q_oud = '''
    SELECT
        submitted_assignment.student_id,
        grade_cell.name AS question_name,
        grade_cell.max_score,
        grade.needs_manual_grade AS needs_grading,
        grade.auto_score,
        grade.manual_score,
        grade.extra_credit,
        comment.auto_comment,
        comment.manual_comment
    FROM grade
        INNER JOIN submitted_notebook ON submitted_notebook.id = grade.notebook_id
        INNER JOIN submitted_assignment ON submitted_assignment.id = submitted_notebook.assignment_id
        INNER JOIN grade_cell ON grade_cell.id = grade.cell_id
        INNER JOIN solution_cell ON solution_cell.name = grade_cell.name AND solution_cell.notebook_id = grade_cell.notebook_id
        LEFT JOIN comment ON comment.notebook_id = grade.notebook_id AND comment.cell_id = solution_cell.id
'''

q = '''
    SELECT
        submitted_assignment.student_id,
        grade_cell.name AS question_name,
        grade_cell.max_score,
        grade.needs_manual_grade AS needs_grading,
        grade.auto_score,
        grade.manual_score,
        grade.extra_credit,
        assignment.name AS assignment
    FROM grade
        INNER JOIN submitted_notebook ON submitted_notebook.id = grade.notebook_id
        INNER JOIN submitted_assignment ON submitted_assignment.id = submitted_notebook.assignment_id
        INNER JOIN grade_cell ON grade_cell.id = grade.cell_id
        INNER JOIN assignment ON submitted_assignment.assignment_id = assignment.id
'''





df = pd.read_sql_query(q, 'sqlite:///gradebook.db')
df = df.set_index([ 'student_id', 'assignment', 'question_name'])
df['final_score'] = np.where(~pd.isnull(df['manual_score']), df['manual_score'], df['auto_score']) + df['extra_credit'].fillna(0)

df.to_csv('student-results.csv')
df.head()

max_score  needs_grading  auto_score manual_score  \
student_id question_name                                                       
1234567    vraag-2               2.0              1         NaN         None   
           vraag-3               1.0              1         NaN         None   
           vraag-1-a-test        1.0              0         0.0         None   
           vraag-1-test          1.0              0         0.0         None   
           vraag_2a-test         1.0              0         0.0         None   

                          extra_credit final_score  
student_id question_name                            
1234567    vraag-2                None         NaN  
           vraag-3                None         NaN  
           vraag-1-a-test         None           0  
           vraag-1-test           None           0  
           vraag_2a-test          None           0

## 8. Analyseren

In [22]:
per_student = df.groupby('student_id')
grades = per_student['final_score'].sum() / per_student['max_score'].sum() * 10

grades

student_id
1234567    0.0
dtype: float64